<a href="https://colab.research.google.com/github/coding-cosmos/Sluice-Gate-Cavitation-Index/blob/main/notebooks/Non_dim_Cavitation_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training


## Utils

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikit-learn
!pip install xgboost
!pip install numpy
!pip install pandas

In [ ]:
!pip install optuna
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.1 MB/s eta 0:00:00


In [ ]:
from sklearn.metrics import r2_score

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import optuna

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import optuna.visualization as vis

In [ ]:
import pandas as pd
import numpy as np


# Load and rename columns
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/non_dim.csv")
df.columns = ['AR', 'phi', 'R/L', 'Fr', 'Qa/Qw', 'sigma']
X = df.drop(columns=['sigma'])
y = df['sigma']


In [ ]:
def split_data(df, target_col='sigma', test_size=0.2, random_state=42):
    X = df.drop(columns=[target_col])
    y = df[target_col]
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

In [ ]:
def train_model(model_cls, X, y, split_ratio=0.2, tune_func=None, model_name="Model", n_trials=30, **kwargs):
    """
    Trains any model with optional Optuna tuning.

    Args:
        model_cls: A model class like GradientBoostingRegressor
        X: Feature dataframe
        y: Target series
        split_ratio: Float ratio for test set
        tune_func: A function accepting (trial, X_train, y_train, X_test, y_test) → rmse
        model_name: Name for logging
        n_trials: Optuna tuning trials (only used if tune_func is provided)
        **kwargs: Default model kwargs if no tuning is done

    Returns:
        model, rmse, y_pred, y_test
    """

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split_ratio, random_state=42)

    if tune_func:
        study = optuna.create_study(direction="minimize")
        study.optimize(lambda trial: tune_func(trial, X_train, y_train, X_test, y_test, model_cls), n_trials=n_trials)
        best_params = study.best_params
        print(f"✅ Best params for {model_name}: {best_params}")
        model = model_cls(**best_params)

        fig = vis.plot_optimization_history(study)
        fig.show()
    else:
        model = model_cls(**kwargs)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"📊 {model_name} RMSE: {rmse:.4f}")

    return {
    "model": model,
    "rmse": rmse,
    "X_train": X_train,
    "X_test": X_test,
    "y_train": y_train,
    "y_test": y_test,
    "y_pred": y_pred
    }



## GBM

In [ ]:
def objective_gbm(trial, X_train, y_train, X_test, y_test,model_cls):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'max_features': trial.suggest_categorical('max_features', [ 'sqrt', 'log2', None])
    }

    model = model_cls(**params)
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse

### 80-20 split

In [ ]:
gbm_20 = train_model(GradientBoostingRegressor,X,y,0.2,objective_gbm,"GBM",n_trials=300)

[I 2025-06-13 18:44:22,047] A new study created in memory with name: no-name-1d856fb8-88ba-4c9a-a619-eaaae15fd769
[I 2025-06-13 18:44:22,137] Trial 0 finished with value: 2.649347421649329 and parameters: {'n_estimators': 68, 'max_depth': 2, 'learning_rate': 0.26377431804277146, 'subsample': 0.7144803424945269, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': 'sqrt'}. Best is trial 0 with value: 2.649347421649329.
[I 2025-06-13 18:44:22,557] Trial 1 finished with value: 8.42917578382341 and parameters: {'n_estimators': 260, 'max_depth': 8, 'learning_rate': 0.006397598043456998, 'subsample': 0.9222209669812912, 'min_samples_split': 5, 'min_samples_leaf': 7, 'max_features': 'sqrt'}. Best is trial 0 with value: 2.649347421649329.
[I 2025-06-13 18:44:22,785] Trial 2 finished with value: 4.794545748664623 and parameters: {'n_estimators': 175, 'max_depth': 7, 'learning_rate': 0.21853163312099796, 'subsample': 0.5393753600896714, 'min_samples_split': 17, 'min_samples_leaf': 16, 

✅ Best params for GBM: {'n_estimators': 252, 'max_depth': 8, 'learning_rate': 0.06285652370947764, 'subsample': 0.8059737929229986, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': None}


📊 GBM RMSE: 0.7292


✅ Best params for GBM: {'n_estimators': 252, 'max_depth': 8, 'learning_rate': 0.06285652370947764, 'subsample': 0.8059737929229986, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': None}


In [ ]:
df_train = gbm_20['X_train'].copy()
df_test = gbm_20['X_test'].copy()

df_train['sigma'] = gbm_20['y_train']
df_test['sigma'] = gbm_20['y_test']


In [ ]:
params_gbm_20 = {'n_estimators': 252, 'max_depth': 8, 'learning_rate': 0.06285652370947764, 'subsample': 0.8059737929229986, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': None}

gbm_20 = train_model(GradientBoostingRegressor,X,y,0.2,None,"GBM",n_trials=0,**params_gbm_20)
df_test['gbm'] = gbm_20['y_pred']
df_train['gbm'] = gbm_20['model'].predict(gbm_20['X_train'])

📊 GBM RMSE: 0.8074


In [ ]:
print(gbm_20['rmse'])
print(r2_score(gbm_20['y_test'],gbm_20['y_pred']))

0.8074183851879887
0.9992149350120249


In [ ]:
df_test

,AR,phi,R/L,Fr,Qa/Qw,sigma,gbm
173,1.666667,10,0.002550,48.788993,4.188136,-1.466537,-2.463901
274,1.666667,30,0.001152,8.565811,0.259974,16.882085,16.762190
489,1.000000,60,0.006268,15.276169,0.850607,-2.385587,-2.699952
72,0.600000,20,0.001742,12.385933,2.270782,3.429377,4.094959
305,1.666667,40,0.001463,17.118945,0.680903,1.905303,1.886948
...,...,...,...,...,...,...,...
331,1.666667,60,0.002008,7.338219,0.000000,11.573187,9.703089
411,1.000000,20,0.001894,42.745852,3.864159,-4.956412,-5.831712
502,1.000000,60,0.002089,13.274464,0.352798,2.171858,2.486488
349,1.000000,10,0.002771,47.151218,5.752433,-4.839180,-4.655865


### 75-25 split

In [ ]:
gbm_25 = train_model(GradientBoostingRegressor,X,y,0.25,objective_gbm,"GBM",n_trials=300)

[I 2025-06-13 18:47:07,165] A new study created in memory with name: no-name-11ab18b3-f15f-4a47-9092-7f174423fe52
[I 2025-06-13 18:47:07,631] Trial 0 finished with value: 23.64728787550284 and parameters: {'n_estimators': 373, 'max_depth': 4, 'learning_rate': 0.000609050799414591, 'subsample': 0.6233150607913129, 'min_samples_split': 16, 'min_samples_leaf': 9, 'max_features': None}. Best is trial 0 with value: 23.64728787550284.
[I 2025-06-13 18:47:07,918] Trial 1 finished with value: 21.322934095301207 and parameters: {'n_estimators': 261, 'max_depth': 4, 'learning_rate': 0.0014356951681675663, 'subsample': 0.6262643429547969, 'min_samples_split': 6, 'min_samples_leaf': 15, 'max_features': None}. Best is trial 1 with value: 21.322934095301207.
[I 2025-06-13 18:47:08,269] Trial 2 finished with value: 15.499058328041706 and parameters: {'n_estimators': 335, 'max_depth': 7, 'learning_rate': 0.002475686684828284, 'subsample': 0.8043313418280617, 'min_samples_split': 13, 'min_samples_leaf'

✅ Best params for GBM: {'n_estimators': 476, 'max_depth': 4, 'learning_rate': 0.1007977370504654, 'subsample': 0.8804670004074493, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': None}


📊 GBM RMSE: 0.6852


In [ ]:
print(gbm_25['rmse'])
print(r2_score(gbm_25['y_test'],gbm_25['y_pred']))

0.685182009716807
0.9994336537853583


### 70-30 split

In [ ]:
gbm_30 = train_model(GradientBoostingRegressor,X,y,0.3,objective_gbm,"GBM",n_trials=300)

[I 2025-06-13 18:50:23,588] A new study created in memory with name: no-name-6f292921-415b-4ba4-bb2a-28af3e48d162
[I 2025-06-13 18:50:23,837] Trial 0 finished with value: 21.7435973599912 and parameters: {'n_estimators': 256, 'max_depth': 5, 'learning_rate': 0.001653765410694861, 'subsample': 0.6462709694188344, 'min_samples_split': 13, 'min_samples_leaf': 10, 'max_features': 'log2'}. Best is trial 0 with value: 21.7435973599912.
[I 2025-06-13 18:50:24,242] Trial 1 finished with value: 7.601799190041793 and parameters: {'n_estimators': 431, 'max_depth': 6, 'learning_rate': 0.012196806145010937, 'subsample': 0.6779517950119056, 'min_samples_split': 16, 'min_samples_leaf': 15, 'max_features': 'log2'}. Best is trial 1 with value: 7.601799190041793.
[I 2025-06-13 18:50:24,612] Trial 2 finished with value: 1.5491265520872641 and parameters: {'n_estimators': 292, 'max_depth': 5, 'learning_rate': 0.011585050400549702, 'subsample': 0.8523669018979353, 'min_samples_split': 4, 'min_samples_leaf'

✅ Best params for GBM: {'n_estimators': 415, 'max_depth': 4, 'learning_rate': 0.09233186100358148, 'subsample': 0.964171620246497, 'min_samples_split': 7, 'min_samples_leaf': 1, 'max_features': None}


📊 GBM RMSE: 0.6404


In [ ]:
print(gbm_30['rmse'])
print(r2_score(gbm_30['y_test'],gbm_30['y_pred']))

0.640406443542444
0.9995264282414908


### 60-40 split

In [ ]:
gbm_40 = train_model(GradientBoostingRegressor,X,y,0.4,objective_gbm,"GBM",n_trials=300)

[I 2025-06-13 18:53:12,346] A new study created in memory with name: no-name-be0933cd-2cc6-4190-83e3-099f674052f2
[I 2025-06-13 18:53:12,749] Trial 0 finished with value: 3.404198250831449 and parameters: {'n_estimators': 396, 'max_depth': 8, 'learning_rate': 0.023842642751773595, 'subsample': 0.7742828426465848, 'min_samples_split': 14, 'min_samples_leaf': 5, 'max_features': 'log2'}. Best is trial 0 with value: 3.404198250831449.
[I 2025-06-13 18:53:13,210] Trial 1 finished with value: 23.527782686715092 and parameters: {'n_estimators': 497, 'max_depth': 5, 'learning_rate': 0.0006108524233834771, 'subsample': 0.9822102614796613, 'min_samples_split': 13, 'min_samples_leaf': 13, 'max_features': 'log2'}. Best is trial 0 with value: 3.404198250831449.
[I 2025-06-13 18:53:13,501] Trial 2 finished with value: 11.029996557392163 and parameters: {'n_estimators': 309, 'max_depth': 6, 'learning_rate': 0.0071465673669520976, 'subsample': 0.9089219722440003, 'min_samples_split': 9, 'min_samples_l

✅ Best params for GBM: {'n_estimators': 348, 'max_depth': 6, 'learning_rate': 0.24469621486898982, 'subsample': 0.9904961900066883, 'min_samples_split': 20, 'min_samples_leaf': 5, 'max_features': None}


📊 GBM RMSE: 1.2756


In [ ]:
print(gbm_40['rmse'])
print(r2_score(gbm_40['y_test'],gbm_40['y_pred']))

1.2756280396843747
0.998072137257753


## XGBoost

In [ ]:
def objective_xgb(trial, X_train, y_train, X_test, y_test,model_cls):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 5)
    }

    model = model_cls(
        **params,
        objective='reg:squarederror',
        verbosity=0,
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse


### 80-20 split

In [ ]:
xgb_20 = train_model(XGBRegressor,X,y,0.2,objective_xgb,"XGBoost",n_trials=300)

[I 2025-06-13 18:55:41,361] A new study created in memory with name: no-name-1d7396f2-8225-4309-aa43-c9c94c25323a
[I 2025-06-13 18:55:41,437] Trial 0 finished with value: 6.848038644137485 and parameters: {'n_estimators': 92, 'max_depth': 3, 'learning_rate': 0.030300148982105403, 'subsample': 0.9131070890973921, 'colsample_bytree': 0.654120328052015, 'gamma': 2.7662049300869618, 'reg_alpha': 3.453348497301258, 'reg_lambda': 3.1206783136458665}. Best is trial 0 with value: 6.848038644137485.
[I 2025-06-13 18:55:41,478] Trial 1 finished with value: 5.541297178599087 and parameters: {'n_estimators': 73, 'max_depth': 5, 'learning_rate': 0.04558273982713097, 'subsample': 0.5844955827370242, 'colsample_bytree': 0.6084460482452041, 'gamma': 3.2045377992798447, 'reg_alpha': 0.027614158363737373, 'reg_lambda': 2.837534258214907}. Best is trial 1 with value: 5.541297178599087.
[I 2025-06-13 18:55:41,569] Trial 2 finished with value: 2.612026774872107 and parameters: {'n_estimators': 419, 'max_de

✅ Best params for XGBoost: {'n_estimators': 374, 'max_depth': 4, 'learning_rate': 0.19506369878658875, 'subsample': 0.7557741773206197, 'colsample_bytree': 0.9008651041744119, 'gamma': 0.31939781060870454, 'reg_alpha': 1.0334200192977028, 'reg_lambda': 1.5791290618713818}


📊 XGBoost RMSE: 1.7544


✅ Best params for XGBoost: {'n_estimators': 374, 'max_depth': 4, 'learning_rate': 0.19506369878658875, 'subsample': 0.7557741773206197, 'colsample_bytree': 0.9008651041744119, 'gamma': 0.31939781060870454, 'reg_alpha': 1.0334200192977028, 'reg_lambda': 1.5791290618713818}


In [ ]:
params_xgb_20 =  {'n_estimators': 374, 'max_depth': 4, 'learning_rate': 0.19506369878658875, 'subsample': 0.7557741773206197, 'colsample_bytree': 0.9008651041744119, 'gamma': 0.31939781060870454, 'reg_alpha': 1.0334200192977028, 'reg_lambda': 1.5791290618713818}
xgb_20 = train_model(XGBRegressor,X,y,0.2,None,"XGBoost",n_trials=0,**params_xgb_20)
df_test['xgb'] = xgb_20['y_pred']
df_train['xgb'] = xgb_20['model'].predict(xgb_20['X_train'])

📊 XGBoost RMSE: 1.7544


In [ ]:
print(xgb_20['rmse'])
print(r2_score(xgb_20['y_test'],xgb_20['y_pred']))

1.7544414302773295
0.9962933115076431


In [ ]:
df_test

,AR,phi,R/L,Fr,Qa/Qw,sigma,gbm,xgb
173,1.666667,10,0.002550,48.788993,4.188136,-1.466537,-2.463901,-2.321761
274,1.666667,30,0.001152,8.565811,0.259974,16.882085,16.762190,21.031942
489,1.000000,60,0.006268,15.276169,0.850607,-2.385587,-2.699952,-2.077010
72,0.600000,20,0.001742,12.385933,2.270782,3.429377,4.094959,4.510517
305,1.666667,40,0.001463,17.118945,0.680903,1.905303,1.886948,1.130121
...,...,...,...,...,...,...,...,...
331,1.666667,60,0.002008,7.338219,0.000000,11.573187,9.703089,13.616969
411,1.000000,20,0.001894,42.745852,3.864159,-4.956412,-5.831712,-5.620557
502,1.000000,60,0.002089,13.274464,0.352798,2.171858,2.486488,3.111056
349,1.000000,10,0.002771,47.151218,5.752433,-4.839180,-4.655865,-5.086569


### 75-25 split

In [ ]:
xgb_25 = train_model(XGBRegressor,X,y,0.25,objective_xgb,"XGBoost",n_trials=300)

[I 2025-06-13 18:57:00,281] A new study created in memory with name: no-name-f65d7655-87dc-4aaf-acc9-97e655429916
[I 2025-06-13 18:57:00,334] Trial 0 finished with value: 17.092197103093568 and parameters: {'n_estimators': 75, 'max_depth': 10, 'learning_rate': 0.015611399709861078, 'subsample': 0.9776720571269546, 'colsample_bytree': 0.577864204724158, 'gamma': 4.333420677597451, 'reg_alpha': 4.186057408982799, 'reg_lambda': 1.5975719079879007}. Best is trial 0 with value: 17.092197103093568.
[I 2025-06-13 18:57:00,467] Trial 1 finished with value: 20.20541089455585 and parameters: {'n_estimators': 333, 'max_depth': 4, 'learning_rate': 0.0015711737269443462, 'subsample': 0.9183440894114979, 'colsample_bytree': 0.7372761339795411, 'gamma': 3.006775352816388, 'reg_alpha': 0.09489605990347649, 'reg_lambda': 0.9799392144955299}. Best is trial 0 with value: 17.092197103093568.
[I 2025-06-13 18:57:00,542] Trial 2 finished with value: 8.087868842653211 and parameters: {'n_estimators': 285, 'm

✅ Best params for XGBoost: {'n_estimators': 286, 'max_depth': 3, 'learning_rate': 0.1695487560167197, 'subsample': 0.5513176418619748, 'colsample_bytree': 0.9993702650786972, 'gamma': 2.0385115890912413, 'reg_alpha': 0.8253588077466615, 'reg_lambda': 0.6336085668680418}


📊 XGBoost RMSE: 1.4347


In [ ]:
print(xgb_25['rmse'])
print(r2_score(xgb_25['y_test'],xgb_25['y_pred']))

1.4347122676706616
0.9975168684802536


### 70-30 split

In [ ]:
xgb_30 = train_model(XGBRegressor,X,y,0.3,objective_xgb,"XGBoost",n_trials=300)

[I 2025-06-13 18:59:13,536] A new study created in memory with name: no-name-446cef09-f806-44b7-8eca-c96d81ec7e65
[I 2025-06-13 18:59:13,592] Trial 0 finished with value: 22.632528678720323 and parameters: {'n_estimators': 190, 'max_depth': 3, 'learning_rate': 0.0025649119850146055, 'subsample': 0.9601226800146199, 'colsample_bytree': 0.5626081862757343, 'gamma': 2.1069207301059305, 'reg_alpha': 0.6028490350764654, 'reg_lambda': 0.8237652145556806}. Best is trial 0 with value: 22.632528678720323.
[I 2025-06-13 18:59:13,635] Trial 1 finished with value: 25.498612673474252 and parameters: {'n_estimators': 92, 'max_depth': 4, 'learning_rate': 0.002234690434687227, 'subsample': 0.8801450838921753, 'colsample_bytree': 0.8894729909618285, 'gamma': 0.8472547263673219, 'reg_alpha': 3.569616609692651, 'reg_lambda': 3.6960980775091903}. Best is trial 0 with value: 22.632528678720323.
[I 2025-06-13 18:59:13,691] Trial 2 finished with value: 27.993821901168037 and parameters: {'n_estimators': 124,

✅ Best params for XGBoost: {'n_estimators': 446, 'max_depth': 3, 'learning_rate': 0.08043462427627233, 'subsample': 0.7604231456487419, 'colsample_bytree': 0.8710609033282802, 'gamma': 0.10663050035935015, 'reg_alpha': 0.6543992943316982, 'reg_lambda': 1.0160528495719312}


📊 XGBoost RMSE: 1.7217


In [ ]:
print(xgb_30['rmse'])
print(r2_score(xgb_30['y_test'],xgb_30['y_pred']))

1.7216870103800042
0.9965771898195607


### 60-40 split

In [ ]:
xgb_40 = train_model(XGBRegressor,X,y,0.4,objective_xgb,"XGBoost",n_trials=300)

[I 2025-06-13 19:00:49,420] A new study created in memory with name: no-name-d6cbc7a6-6281-4dd5-a948-fc9e1edaab7e
[I 2025-06-13 19:00:49,505] Trial 0 finished with value: 3.362920979394122 and parameters: {'n_estimators': 369, 'max_depth': 7, 'learning_rate': 0.2686252054544, 'subsample': 0.5049580298176146, 'colsample_bytree': 0.8324497493314252, 'gamma': 2.907672594739817, 'reg_alpha': 2.1663692203513008, 'reg_lambda': 4.1546262320213}. Best is trial 0 with value: 3.362920979394122.
[I 2025-06-13 19:00:49,645] Trial 1 finished with value: 5.4228122321526895 and parameters: {'n_estimators': 353, 'max_depth': 8, 'learning_rate': 0.03644068585624711, 'subsample': 0.8238351678324144, 'colsample_bytree': 0.5216592105295361, 'gamma': 3.7431172739894736, 'reg_alpha': 2.0066979331756887, 'reg_lambda': 1.960333733542159}. Best is trial 0 with value: 3.362920979394122.
[I 2025-06-13 19:00:49,760] Trial 2 finished with value: 3.8387522102303264 and parameters: {'n_estimators': 423, 'max_depth':

✅ Best params for XGBoost: {'n_estimators': 481, 'max_depth': 3, 'learning_rate': 0.12944223859466703, 'subsample': 0.8128922369047058, 'colsample_bytree': 0.8214074142414485, 'gamma': 0.26608708297815786, 'reg_alpha': 0.309505713121324, 'reg_lambda': 1.0000098240255224}


📊 XGBoost RMSE: 2.6085


In [ ]:
print(xgb_40['rmse'])
print(r2_score(xgb_40['y_test'],xgb_40['y_pred']))

2.6085085253971605
0.9919385620162733


## CatBoost

In [ ]:
def objective_catb(trial, X_train, y_train, X_test, y_test,model_cls):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 500),
        'depth': trial.suggest_int('depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.3, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 1.0)
    }

    model = model_cls(
        **params,
        loss_function='RMSE',
        verbose=0,
        random_seed=42
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse


### 80-20 split

In [ ]:
catb_20 = train_model(CatBoostRegressor,X,y,0.2,objective_catb,"CatBoost",n_trials=300)

[I 2025-06-13 19:03:25,966] A new study created in memory with name: no-name-f181e06e-d971-4a21-87e3-889501aac0d4
[I 2025-06-13 19:03:26,851] Trial 0 finished with value: 23.10019163810782 and parameters: {'iterations': 366, 'depth': 7, 'learning_rate': 0.0009784623495872917, 'l2_leaf_reg': 8.927474513862478, 'bagging_temperature': 0.48805835556095833, 'random_strength': 0.049188503610684164}. Best is trial 0 with value: 23.10019163810782.
[I 2025-06-13 19:03:30,031] Trial 1 finished with value: 25.27468304558104 and parameters: {'iterations': 483, 'depth': 10, 'learning_rate': 0.0004151430308625116, 'l2_leaf_reg': 2.6190435935551037, 'bagging_temperature': 0.37587895738379673, 'random_strength': 0.28758525560720616}. Best is trial 0 with value: 23.10019163810782.
[I 2025-06-13 19:03:30,217] Trial 2 finished with value: 28.107368449391338 and parameters: {'iterations': 321, 'depth': 5, 'learning_rate': 0.00013035085436387855, 'l2_leaf_reg': 9.943098658463455, 'bagging_temperature': 0.8

✅ Best params for CatBoost: {'iterations': 441, 'depth': 6, 'learning_rate': 0.15310299327795915, 'l2_leaf_reg': 3.3330519675173056, 'bagging_temperature': 0.11689119833947537, 'random_strength': 0.000706267657318276}


0:	learn: 22.6999395	total: 1.08ms	remaining: 474ms
1:	learn: 19.9584198	total: 1.86ms	remaining: 409ms
2:	learn: 17.6840043	total: 2.6ms	remaining: 380ms
3:	learn: 15.7514310	total: 3.31ms	remaining: 361ms
4:	learn: 13.9755036	total: 4ms	remaining: 348ms
5:	learn: 12.4013252	total: 4.66ms	remaining: 338ms
6:	learn: 11.0409741	total: 5.37ms	remaining: 333ms
7:	learn: 9.8505996	total: 6.05ms	remaining: 328ms
8:	learn: 8.7909677	total: 6.74ms	remaining: 324ms
9:	learn: 7.8347241	total: 7.43ms	remaining: 320ms
10:	learn: 7.0166902	total: 8.11ms	remaining: 317ms
11:	learn: 6.3130356	total: 8.78ms	remaining: 314ms
12:	learn: 5.6984426	total: 9.43ms	remaining: 311ms
13:	learn: 5.1650305	total: 10.1ms	remaining: 308ms
14:	learn: 4.6756811	total: 10.8ms	remaining: 306ms
15:	learn: 4.2949956	total: 11.5ms	remaining: 305ms
16:	learn: 3.9536667	total: 12.1ms	remaining: 302ms
17:	learn: 3.6529555	total: 12.8ms	remaining: 301ms
18:	learn: 3.3907913	total: 13.5ms	remaining: 300ms
19:	learn: 3.179894

✅ Best params for CatBoost: {'iterations': 441, 'depth': 6, 'learning_rate': 0.15310299327795915, 'l2_leaf_reg': 3.3330519675173056, 'bagging_temperature': 0.11689119833947537, 'random_strength': 0.000706267657318276}


In [ ]:
params_Cat_20 = {'iterations': 441, 'depth': 6, 'learning_rate': 0.15310299327795915, 'l2_leaf_reg': 3.3330519675173056, 'bagging_temperature': 0.11689119833947537, 'random_strength': 0.000706267657318276}
Cat_20 = train_model(CatBoostRegressor,X,y,0.2,None,"CatBoost",n_trials=0,**params_Cat_20)
df_test['Cat'] = Cat_20['y_pred']
df_train['Cat'] = Cat_20['model'].predict(Cat_20['X_train'])

0:	learn: 22.6999395	total: 47.9ms	remaining: 21.1s
1:	learn: 19.9584198	total: 48.7ms	remaining: 10.7s
2:	learn: 17.6840043	total: 49.5ms	remaining: 7.22s
3:	learn: 15.7514310	total: 50.3ms	remaining: 5.5s
4:	learn: 13.9755036	total: 51ms	remaining: 4.44s
5:	learn: 12.4013252	total: 51.6ms	remaining: 3.74s
6:	learn: 11.0409741	total: 52.3ms	remaining: 3.24s
7:	learn: 9.8505996	total: 53ms	remaining: 2.87s
8:	learn: 8.7909677	total: 53.8ms	remaining: 2.58s
9:	learn: 7.8347241	total: 54.5ms	remaining: 2.35s
10:	learn: 7.0166902	total: 55.1ms	remaining: 2.15s
11:	learn: 6.3130356	total: 55.9ms	remaining: 2s
12:	learn: 5.6984426	total: 56.6ms	remaining: 1.86s
13:	learn: 5.1650305	total: 57.3ms	remaining: 1.75s
14:	learn: 4.6756811	total: 58ms	remaining: 1.65s
15:	learn: 4.2949956	total: 58.7ms	remaining: 1.56s
16:	learn: 3.9536667	total: 59.4ms	remaining: 1.48s
17:	learn: 3.6529555	total: 60.1ms	remaining: 1.41s
18:	learn: 3.3907913	total: 60.7ms	remaining: 1.35s
19:	learn: 3.1798941	tota

In [ ]:
print(Cat_20['rmse'])
print(r2_score(Cat_20['y_test'],Cat_20['y_pred']))

0.6318259252470015
0.9995192677864441


In [ ]:
df_test

,AR,phi,R/L,Fr,Qa/Qw,sigma,gbm,xgb,Cat
173,1.666667,10,0.002550,48.788993,4.188136,-1.466537,-2.463901,-2.321761,-2.381526
274,1.666667,30,0.001152,8.565811,0.259974,16.882085,16.762190,21.031942,16.787864
489,1.000000,60,0.006268,15.276169,0.850607,-2.385587,-2.699952,-2.077010,-2.281170
72,0.600000,20,0.001742,12.385933,2.270782,3.429377,4.094959,4.510517,3.545761
305,1.666667,40,0.001463,17.118945,0.680903,1.905303,1.886948,1.130121,1.095775
...,...,...,...,...,...,...,...,...,...
331,1.666667,60,0.002008,7.338219,0.000000,11.573187,9.703089,13.616969,10.833187
411,1.000000,20,0.001894,42.745852,3.864159,-4.956412,-5.831712,-5.620557,-5.481635
502,1.000000,60,0.002089,13.274464,0.352798,2.171858,2.486488,3.111056,2.352765
349,1.000000,10,0.002771,47.151218,5.752433,-4.839180,-4.655865,-5.086569,-4.850669


### 75-25 split

In [ ]:
catb_25 = train_model(CatBoostRegressor,X,y,0.25,objective_catb,"XGBoost",n_trials=300)

[I 2025-06-13 19:06:49,137] A new study created in memory with name: no-name-907b0aa1-fb69-428d-9ba7-4e32dbcf6f24
[I 2025-06-13 19:06:50,220] Trial 0 finished with value: 28.263369140210475 and parameters: {'iterations': 348, 'depth': 9, 'learning_rate': 0.00011974687900922985, 'l2_leaf_reg': 9.796132539188045, 'bagging_temperature': 0.9373686363268042, 'random_strength': 0.40716922344746453}. Best is trial 0 with value: 28.263369140210475.
[I 2025-06-13 19:06:50,804] Trial 1 finished with value: 3.5081760959845014 and parameters: {'iterations': 278, 'depth': 8, 'learning_rate': 0.21348670265067282, 'l2_leaf_reg': 9.647969437522162, 'bagging_temperature': 0.9196212352138449, 'random_strength': 0.3162439527338684}. Best is trial 1 with value: 3.5081760959845014.
[I 2025-06-13 19:06:50,982] Trial 2 finished with value: 2.323340288508591 and parameters: {'iterations': 286, 'depth': 5, 'learning_rate': 0.0809718296410884, 'l2_leaf_reg': 7.287574186028929, 'bagging_temperature': 0.462171808

✅ Best params for XGBoost: {'iterations': 393, 'depth': 6, 'learning_rate': 0.18045676674940558, 'l2_leaf_reg': 7.97108769833924, 'bagging_temperature': 0.3786846435986671, 'random_strength': 0.035990839162005156}


0:	learn: 22.8127270	total: 2.22ms	remaining: 871ms
1:	learn: 20.2380976	total: 3.82ms	remaining: 746ms
2:	learn: 17.7780225	total: 4.8ms	remaining: 624ms
3:	learn: 15.7058594	total: 5.74ms	remaining: 558ms
4:	learn: 14.0597026	total: 6.68ms	remaining: 519ms
5:	learn: 12.6159151	total: 12.1ms	remaining: 778ms
6:	learn: 11.5103142	total: 13ms	remaining: 718ms
7:	learn: 10.4621975	total: 14ms	remaining: 676ms
8:	learn: 9.4692504	total: 14.8ms	remaining: 633ms
9:	learn: 8.6400513	total: 16.9ms	remaining: 647ms
10:	learn: 7.9463251	total: 17.9ms	remaining: 621ms
11:	learn: 7.2406055	total: 18.9ms	remaining: 600ms
12:	learn: 6.5861762	total: 19.9ms	remaining: 582ms
13:	learn: 6.0291121	total: 20.9ms	remaining: 565ms
14:	learn: 5.4936239	total: 21.9ms	remaining: 552ms
15:	learn: 5.0513097	total: 22.9ms	remaining: 539ms
16:	learn: 4.6668715	total: 28.8ms	remaining: 637ms
17:	learn: 4.3244991	total: 29.9ms	remaining: 622ms
18:	learn: 4.0536789	total: 30.9ms	remaining: 608ms
19:	learn: 3.829818

In [ ]:
print(catb_25['rmse'])
print(r2_score(catb_25['y_test'],catb_25['y_pred']))

1.3823048343686435
0.9976949637862096


### 70-30 split

In [ ]:
catb_30 = train_model(CatBoostRegressor,X,y,0.3,objective_catb,"XGBoost",n_trials=300)

[I 2025-06-13 19:09:59,481] A new study created in memory with name: no-name-51ebdfe7-8dbf-440e-abb1-12ff402a4633
[I 2025-06-13 19:10:00,949] Trial 0 finished with value: 3.890675225744378 and parameters: {'iterations': 218, 'depth': 10, 'learning_rate': 0.0630523453924496, 'l2_leaf_reg': 1.2461581283945642, 'bagging_temperature': 0.932781162237911, 'random_strength': 0.22984739116006647}. Best is trial 0 with value: 3.890675225744378.
[I 2025-06-13 19:10:01,415] Trial 1 finished with value: 14.006676888152366 and parameters: {'iterations': 228, 'depth': 8, 'learning_rate': 0.005241809181773524, 'l2_leaf_reg': 4.11580181612028, 'bagging_temperature': 0.23446488568871526, 'random_strength': 0.08234154074099986}. Best is trial 0 with value: 3.890675225744378.
[I 2025-06-13 19:10:04,410] Trial 2 finished with value: 3.237085114532277 and parameters: {'iterations': 458, 'depth': 10, 'learning_rate': 0.046664656515357825, 'l2_leaf_reg': 5.094507304731593, 'bagging_temperature': 0.7306303007

✅ Best params for XGBoost: {'iterations': 483, 'depth': 4, 'learning_rate': 0.21491805949949175, 'l2_leaf_reg': 8.95180350663598, 'bagging_temperature': 0.5244998813895007, 'random_strength': 0.8322707850193969}


0:	learn: 22.3239767	total: 414us	remaining: 200ms
1:	learn: 20.4496845	total: 779us	remaining: 187ms
2:	learn: 17.8807444	total: 1.09ms	remaining: 174ms
3:	learn: 16.2030178	total: 1.42ms	remaining: 170ms
4:	learn: 14.7989235	total: 1.72ms	remaining: 164ms
5:	learn: 13.5922522	total: 2.04ms	remaining: 162ms
6:	learn: 12.2015592	total: 2.36ms	remaining: 161ms
7:	learn: 10.8251160	total: 2.66ms	remaining: 158ms
8:	learn: 9.7908386	total: 3.01ms	remaining: 158ms
9:	learn: 9.0851044	total: 3.31ms	remaining: 157ms
10:	learn: 8.1551165	total: 3.61ms	remaining: 155ms
11:	learn: 7.5705340	total: 3.94ms	remaining: 155ms
12:	learn: 6.9975136	total: 4.24ms	remaining: 153ms
13:	learn: 6.4988007	total: 4.54ms	remaining: 152ms
14:	learn: 6.0108731	total: 4.88ms	remaining: 152ms
15:	learn: 5.7271428	total: 5.17ms	remaining: 151ms
16:	learn: 5.5146704	total: 5.48ms	remaining: 150ms
17:	learn: 5.2619097	total: 5.78ms	remaining: 149ms
18:	learn: 4.9755000	total: 6.08ms	remaining: 149ms
19:	learn: 4.676

In [ ]:
print(catb_30['rmse'])
print(r2_score(catb_30['y_test'],catb_30['y_pred']))

1.2194836402993836
0.9982827783060756


### 60-40 split

In [ ]:
catb_40 = train_model(CatBoostRegressor,X,y,0.4,objective_catb,"XGBoost",n_trials=300)

[I 2025-06-13 19:12:06,495] A new study created in memory with name: no-name-ed179d4e-0dbf-43ab-8c3d-48bf0dd2abed
[I 2025-06-13 19:12:06,737] Trial 0 finished with value: 25.615942636639897 and parameters: {'iterations': 310, 'depth': 6, 'learning_rate': 0.0008646635090473544, 'l2_leaf_reg': 9.178387437732622, 'bagging_temperature': 0.1361816818148417, 'random_strength': 0.42481565488259876}. Best is trial 0 with value: 25.615942636639897.
[I 2025-06-13 19:12:07,688] Trial 1 finished with value: 9.582796186752233 and parameters: {'iterations': 307, 'depth': 9, 'learning_rate': 0.007420379739641918, 'l2_leaf_reg': 1.8159748031527394, 'bagging_temperature': 0.5015563452731291, 'random_strength': 0.8569406150929588}. Best is trial 1 with value: 9.582796186752233.
[I 2025-06-13 19:12:07,885] Trial 2 finished with value: 2.9536645656453486 and parameters: {'iterations': 335, 'depth': 5, 'learning_rate': 0.11331633070222576, 'l2_leaf_reg': 9.560079474376714, 'bagging_temperature': 0.86800031

✅ Best params for XGBoost: {'iterations': 241, 'depth': 6, 'learning_rate': 0.092912733020211, 'l2_leaf_reg': 1.5202796988835932, 'bagging_temperature': 0.09418053385681704, 'random_strength': 0.0019403638987619005}


0:	learn: 22.5850007	total: 819us	remaining: 197ms
1:	learn: 20.8559274	total: 1.82ms	remaining: 217ms
2:	learn: 19.3184227	total: 2.53ms	remaining: 201ms
3:	learn: 17.9049897	total: 3.21ms	remaining: 191ms
4:	learn: 16.5740623	total: 3.87ms	remaining: 183ms
5:	learn: 15.3470416	total: 4.53ms	remaining: 177ms
6:	learn: 14.2141353	total: 5.19ms	remaining: 173ms
7:	learn: 13.1752924	total: 5.85ms	remaining: 170ms
8:	learn: 12.2653862	total: 6.5ms	remaining: 168ms
9:	learn: 11.4229233	total: 7.17ms	remaining: 166ms
10:	learn: 10.6662822	total: 7.85ms	remaining: 164ms
11:	learn: 9.9266978	total: 8.51ms	remaining: 162ms
12:	learn: 9.2465332	total: 9.16ms	remaining: 161ms
13:	learn: 8.6074684	total: 9.82ms	remaining: 159ms
14:	learn: 8.0261533	total: 10.5ms	remaining: 158ms
15:	learn: 7.4710226	total: 11.1ms	remaining: 157ms
16:	learn: 6.9862790	total: 11.8ms	remaining: 156ms
17:	learn: 6.5108917	total: 12.5ms	remaining: 155ms
18:	learn: 6.0853433	total: 13.1ms	remaining: 153ms
19:	learn: 5.

In [ ]:
print(catb_40['rmse'])
print(r2_score(catb_40['y_test'],catb_40['y_pred']))

2.514070392960996
0.9925117064042884


## XRT

In [ ]:
def objective_xrt(trial, X_train, y_train, X_test, y_test,model_cls):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
    }

    model = model_cls(**params, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse


### 80-20 split

In [ ]:
xrt_20 = train_model(ExtraTreesRegressor,X,y,0.2,objective_xrt,"XRT",n_trials=300)

[I 2025-06-13 19:14:18,650] A new study created in memory with name: no-name-b52558a5-f67c-4aba-a041-f8c26d6583c5
[I 2025-06-13 19:14:18,990] Trial 0 finished with value: 14.260626185272148 and parameters: {'n_estimators': 234, 'max_depth': 6, 'min_samples_split': 8, 'min_samples_leaf': 9, 'max_features': 'log2'}. Best is trial 0 with value: 14.260626185272148.
[I 2025-06-13 19:14:19,280] Trial 1 finished with value: 15.628835299938066 and parameters: {'n_estimators': 200, 'max_depth': 4, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt'}. Best is trial 0 with value: 14.260626185272148.
[I 2025-06-13 19:14:19,502] Trial 2 finished with value: 13.222934136553347 and parameters: {'n_estimators': 151, 'max_depth': 12, 'min_samples_split': 5, 'min_samples_leaf': 9, 'max_features': 'log2'}. Best is trial 2 with value: 13.222934136553347.
[I 2025-06-13 19:14:19,874] Trial 3 finished with value: 3.923580333547515 and parameters: {'n_estimators': 262, 'max_depth': 6, 'min_s

✅ Best params for XRT: {'n_estimators': 495, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None}


📊 XRT RMSE: 0.7411


✅ Best params for XRT: {'n_estimators': 495, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None}


In [ ]:
params_XRT_20 = {'n_estimators': 495, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None}
XRT_20 = train_model(ExtraTreesRegressor,X,y,0.2,None,"CatBoost",n_trials=0,**params_XRT_20)
df_test['XRT'] = XRT_20['y_pred']
df_train['XRT'] = XRT_20['model'].predict(XRT_20['X_train'])

📊 CatBoost RMSE: 0.7625


In [ ]:
print(XRT_20['rmse'])
print(r2_score(XRT_20['y_test'],XRT_20['y_pred']))

0.762469055294531
0.9992999117542658


In [ ]:
df_test

,AR,phi,R/L,Fr,Qa/Qw,sigma,gbm,xgb,Cat,XRT
173,1.666667,10,0.002550,48.788993,4.188136,-1.466537,-2.463901,-2.321761,-2.381526,-1.577516
274,1.666667,30,0.001152,8.565811,0.259974,16.882085,16.762190,21.031942,16.787864,16.565363
489,1.000000,60,0.006268,15.276169,0.850607,-2.385587,-2.699952,-2.077010,-2.281170,-1.722313
72,0.600000,20,0.001742,12.385933,2.270782,3.429377,4.094959,4.510517,3.545761,4.162998
305,1.666667,40,0.001463,17.118945,0.680903,1.905303,1.886948,1.130121,1.095775,1.571805
...,...,...,...,...,...,...,...,...,...,...
331,1.666667,60,0.002008,7.338219,0.000000,11.573187,9.703089,13.616969,10.833187,11.502864
411,1.000000,20,0.001894,42.745852,3.864159,-4.956412,-5.831712,-5.620557,-5.481635,-5.447142
502,1.000000,60,0.002089,13.274464,0.352798,2.171858,2.486488,3.111056,2.352765,1.695021
349,1.000000,10,0.002771,47.151218,5.752433,-4.839180,-4.655865,-5.086569,-4.850669,-4.332065


### 75-25 split

In [ ]:
xrt_25 = train_model(ExtraTreesRegressor,X,y,0.25,objective_xrt,"XRT",n_trials=300)

[I 2025-06-13 19:18:38,851] A new study created in memory with name: no-name-bcb6c318-8a50-4565-a547-3f053f6fbdb9
[I 2025-06-13 19:18:39,326] Trial 0 finished with value: 11.741702126648391 and parameters: {'n_estimators': 262, 'max_depth': 3, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': None}. Best is trial 0 with value: 11.741702126648391.
[I 2025-06-13 19:18:40,244] Trial 1 finished with value: 7.867105028309816 and parameters: {'n_estimators': 455, 'max_depth': 18, 'min_samples_split': 9, 'min_samples_leaf': 2, 'max_features': 'sqrt'}. Best is trial 1 with value: 7.867105028309816.
[I 2025-06-13 19:18:40,783] Trial 2 finished with value: 15.697660792994975 and parameters: {'n_estimators': 378, 'max_depth': 4, 'min_samples_split': 3, 'min_samples_leaf': 5, 'max_features': 'log2'}. Best is trial 1 with value: 7.867105028309816.
[I 2025-06-13 19:18:41,093] Trial 3 finished with value: 13.656190717113393 and parameters: {'n_estimators': 215, 'max_depth': 16, 'min_samp

✅ Best params for XRT: {'n_estimators': 332, 'max_depth': 12, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None}


📊 XRT RMSE: 1.2697


In [ ]:
print(xrt_25['rmse'])
print(r2_score(xrt_25['y_test'],xrt_25['y_pred']))

1.269707314582301
0.9980551892694535


### 70-30 split

In [ ]:
xrt_30 = train_model(ExtraTreesRegressor,X,y,0.3,objective_xrt,"XRT",n_trials=300)

[I 2025-06-13 19:21:38,252] A new study created in memory with name: no-name-f7be58c2-7c22-429b-9c36-5adf9d93bbdd
[I 2025-06-13 19:21:38,452] Trial 0 finished with value: 14.005012156146588 and parameters: {'n_estimators': 100, 'max_depth': 15, 'min_samples_split': 6, 'min_samples_leaf': 8, 'max_features': 'sqrt'}. Best is trial 0 with value: 14.005012156146588.
[I 2025-06-13 19:21:38,692] Trial 1 finished with value: 14.24360936373242 and parameters: {'n_estimators': 119, 'max_depth': 8, 'min_samples_split': 2, 'min_samples_leaf': 8, 'max_features': 'sqrt'}. Best is trial 0 with value: 14.005012156146588.
[I 2025-06-13 19:21:39,433] Trial 2 finished with value: 4.035577005421498 and parameters: {'n_estimators': 350, 'max_depth': 7, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': None}. Best is trial 2 with value: 4.035577005421498.
[I 2025-06-13 19:21:39,600] Trial 3 finished with value: 13.855447608980706 and parameters: {'n_estimators': 107, 'max_depth': 17, 'min_sam

✅ Best params for XRT: {'n_estimators': 423, 'max_depth': 13, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None}


📊 XRT RMSE: 0.9274


In [ ]:
print(xrt_30['rmse'])
print(r2_score(xrt_30['y_test'],xrt_30['y_pred']))

0.9274186503701318
0.9990068243576603


### 60-40 split

In [ ]:
xrt_40 = train_model(ExtraTreesRegressor,X,y,0.4,objective_xrt,"XRT",n_trials=300)

[I 2025-06-13 19:25:23,590] A new study created in memory with name: no-name-946a38b1-194a-4d01-8263-8361257bf7d2
[I 2025-06-13 19:25:24,202] Trial 0 finished with value: 8.696717049395602 and parameters: {'n_estimators': 400, 'max_depth': 11, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2'}. Best is trial 0 with value: 8.696717049395602.
[I 2025-06-13 19:25:24,458] Trial 1 finished with value: 15.625428136281679 and parameters: {'n_estimators': 173, 'max_depth': 17, 'min_samples_split': 6, 'min_samples_leaf': 9, 'max_features': 'sqrt'}. Best is trial 0 with value: 8.696717049395602.
[I 2025-06-13 19:25:24,894] Trial 2 finished with value: 9.883577194890366 and parameters: {'n_estimators': 293, 'max_depth': 17, 'min_samples_split': 10, 'min_samples_leaf': 9, 'max_features': None}. Best is trial 0 with value: 8.696717049395602.
[I 2025-06-13 19:25:25,372] Trial 3 finished with value: 10.243880805162227 and parameters: {'n_estimators': 308, 'max_depth': 8, 'min_samp

✅ Best params for XRT: {'n_estimators': 209, 'max_depth': 16, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None}


📊 XRT RMSE: 1.5256


In [ ]:
print(xrt_40['rmse'])
print(r2_score(xrt_40['y_test'],xrt_40['y_pred']))

1.5256441195029147
0.9972423796506095


## Random Forest

In [ ]:
def objective_rf(trial, X_train, y_train, X_test, y_test,model_cls):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
    }

    model = model_cls(**params, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse


### 80-20 split

In [ ]:
rf_20 = train_model(RandomForestRegressor,X,y,0.2,objective_rf,"RF",n_trials=300)

[I 2025-06-13 19:28:27,252] A new study created in memory with name: no-name-cf4418c2-6d6f-4e39-8652-cb18b78c5238
[I 2025-06-13 19:28:28,465] Trial 0 finished with value: 8.9826010682304 and parameters: {'n_estimators': 483, 'max_depth': 9, 'min_samples_split': 3, 'min_samples_leaf': 9, 'max_features': 'log2'}. Best is trial 0 with value: 8.9826010682304.
[I 2025-06-13 19:28:29,256] Trial 1 finished with value: 9.006447994710346 and parameters: {'n_estimators': 417, 'max_depth': 9, 'min_samples_split': 5, 'min_samples_leaf': 9, 'max_features': 'sqrt'}. Best is trial 0 with value: 8.9826010682304.
[I 2025-06-13 19:28:29,880] Trial 2 finished with value: 5.037477313157731 and parameters: {'n_estimators': 319, 'max_depth': 14, 'min_samples_split': 7, 'min_samples_leaf': 1, 'max_features': 'log2'}. Best is trial 2 with value: 5.037477313157731.
[I 2025-06-13 19:28:30,590] Trial 3 finished with value: 7.335854744842047 and parameters: {'n_estimators': 361, 'max_depth': 20, 'min_samples_spli

✅ Best params for RF: {'n_estimators': 160, 'max_depth': 17, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': None}


📊 RF RMSE: 1.2864


✅ Best params for RF: {'n_estimators': 160, 'max_depth': 17, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': None}


In [ ]:
params_RF_20 = {'n_estimators': 160, 'max_depth': 17, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': None}

RF_20 = train_model(RandomForestRegressor,X,y,0.2,None,"CatBoost",n_trials=0,**params_RF_20)
df_test['RF'] = RF_20['y_pred']
df_train['RF'] = RF_20['model'].predict(RF_20['X_train'])

📊 CatBoost RMSE: 1.3299


In [ ]:
print(RF_20['rmse'])
print(r2_score(RF_20['y_test'],RF_20['y_pred']))

1.3298546000476112
0.997870308681113


In [ ]:
df_train

,AR,phi,R/L,Fr,Qa/Qw,sigma,gbm,xgb,Cat,XRT,RF
75,0.600000,20,0.001742,24.771866,3.481277,-6.644326,-6.473817,-6.242246,-6.422716,-6.635390,-6.034464
442,1.000000,30,0.001518,7.297885,0.737960,17.974254,17.994039,17.851439,18.204391,17.973463,18.008553
15,0.600000,10,0.001000,12.326323,2.368906,13.166125,13.153822,13.226031,13.174457,13.166194,12.875244
325,1.666667,60,0.003012,9.784292,0.307673,5.919795,6.045507,6.109040,5.997033,5.919770,6.623180
388,1.000000,15,0.001109,21.390516,4.345285,-3.789019,-3.762751,-3.395911,-3.695086,-3.791305,-3.203209
...,...,...,...,...,...,...,...,...,...,...,...
106,0.600000,30,0.001543,6.242637,0.127235,18.348362,18.263476,17.979401,18.429496,18.348364,19.199869
270,1.666667,30,0.001728,21.414526,2.342828,-5.836333,-5.685199,-5.112775,-5.570854,-5.843933,-5.482710
348,1.000000,10,0.002771,39.846099,5.733505,-4.384864,-4.323267,-4.522866,-4.419546,-4.384471,-4.114034
435,1.000000,30,0.002277,7.297885,0.839748,17.717556,17.788899,18.134050,17.635275,17.721635,17.932382


### 75-25 split

In [ ]:
rf_25 = train_model(RandomForestRegressor,X,y,0.25,objective_rf,"RF",n_trials=300)

[I 2025-06-13 19:31:27,956] A new study created in memory with name: no-name-9ab1765c-fc3b-45fe-bd1e-1ba8056e5d19
[I 2025-06-13 19:31:28,857] Trial 0 finished with value: 8.553249722559844 and parameters: {'n_estimators': 481, 'max_depth': 18, 'min_samples_split': 6, 'min_samples_leaf': 8, 'max_features': 'sqrt'}. Best is trial 0 with value: 8.553249722559844.
[I 2025-06-13 19:31:29,600] Trial 1 finished with value: 3.876136271593649 and parameters: {'n_estimators': 363, 'max_depth': 12, 'min_samples_split': 5, 'min_samples_leaf': 7, 'max_features': None}. Best is trial 1 with value: 3.876136271593649.
[I 2025-06-13 19:31:29,951] Trial 2 finished with value: 9.030808260668364 and parameters: {'n_estimators': 178, 'max_depth': 3, 'min_samples_split': 7, 'min_samples_leaf': 4, 'max_features': 'sqrt'}. Best is trial 1 with value: 3.876136271593649.
[I 2025-06-13 19:31:30,356] Trial 3 finished with value: 6.7837919419553 and parameters: {'n_estimators': 211, 'max_depth': 5, 'min_samples_sp

✅ Best params for RF: {'n_estimators': 100, 'max_depth': 11, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None}


📊 RF RMSE: 1.4686


In [ ]:
print(rf_25['rmse'])
print(r2_score(rf_25['y_test'],rf_25['y_pred']))

1.4686362744310353
0.9973980520731882


### 70-30 split

In [ ]:
rf_30 = train_model(RandomForestRegressor,X,y,0.3,objective_rf,"RF",n_trials=300)

[I 2025-06-13 19:36:46,451] A new study created in memory with name: no-name-bed0f183-c66e-419c-97aa-77260806eb24
[I 2025-06-13 19:36:46,759] Trial 0 finished with value: 7.941712405964398 and parameters: {'n_estimators': 144, 'max_depth': 19, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': 'sqrt'}. Best is trial 0 with value: 7.941712405964398.
[I 2025-06-13 19:36:47,532] Trial 1 finished with value: 9.16517763140231 and parameters: {'n_estimators': 397, 'max_depth': 12, 'min_samples_split': 9, 'min_samples_leaf': 7, 'max_features': 'sqrt'}. Best is trial 0 with value: 7.941712405964398.
[I 2025-06-13 19:36:48,002] Trial 2 finished with value: 4.602487882534379 and parameters: {'n_estimators': 238, 'max_depth': 4, 'min_samples_split': 7, 'min_samples_leaf': 6, 'max_features': None}. Best is trial 2 with value: 4.602487882534379.
[I 2025-06-13 19:36:48,938] Trial 3 finished with value: 9.102100215119105 and parameters: {'n_estimators': 485, 'max_depth': 20, 'min_samples_

✅ Best params for RF: {'n_estimators': 290, 'max_depth': 17, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None}


📊 RF RMSE: 1.6658


In [ ]:
print(rf_30['rmse'])
print(r2_score(rf_30['y_test'],rf_30['y_pred']))

1.6658385967579687
0.9967956477613552


### 60-40 split

In [ ]:
rf_40 = train_model(RandomForestRegressor,X,y,0.4,objective_rf,"RF",n_trials=300)

[I 2025-06-13 19:41:22,730] A new study created in memory with name: no-name-efdf3285-92b4-4be4-9383-d3e5133b6632
[I 2025-06-13 19:41:23,433] Trial 0 finished with value: 4.9726912941931065 and parameters: {'n_estimators': 327, 'max_depth': 13, 'min_samples_split': 9, 'min_samples_leaf': 2, 'max_features': None}. Best is trial 0 with value: 4.9726912941931065.
[I 2025-06-13 19:41:23,738] Trial 1 finished with value: 10.773958449111506 and parameters: {'n_estimators': 149, 'max_depth': 20, 'min_samples_split': 7, 'min_samples_leaf': 7, 'max_features': 'log2'}. Best is trial 0 with value: 4.9726912941931065.
[I 2025-06-13 19:41:24,498] Trial 2 finished with value: 4.499148846455164 and parameters: {'n_estimators': 357, 'max_depth': 10, 'min_samples_split': 7, 'min_samples_leaf': 1, 'max_features': None}. Best is trial 2 with value: 4.499148846455164.
[I 2025-06-13 19:41:25,258] Trial 3 finished with value: 5.814790663631288 and parameters: {'n_estimators': 371, 'max_depth': 14, 'min_samp

✅ Best params for RF: {'n_estimators': 151, 'max_depth': 20, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': None}


📊 RF RMSE: 4.6497


In [ ]:
print(rf_40['rmse'])
print(r2_score(rf_40['y_test'],rf_40['y_pred']))

4.649696224152878
0.9743860146366706


## Save Files


In [ ]:
df_test.to_csv('ndim_testing.csv',index=False)
df_train.to_csv('ndim_training.csv',index=False)